In [33]:
import pandas as pd
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Embedding, Dense, BatchNormalization, Dropout, Input
from keras.optimizers import SGD
from keras.utils import to_categorical

In [2]:
def rebalance(frame, col='hab_lbl', factor=1):
    max_size = frame[col].value_counts().max()
    lst = [frame]
    for class_index, group in frame.groupby(col):
        lst.append(group.sample(int((max_size-len(group)) / factor), replace=True))
    frame_new = pd.concat(lst)
    
    return frame_new

In [3]:
df = pd.read_csv('dataset-rocky-no-STemp.csv')
df = df.drop('P. Habitable', axis=1)
y = df['hab_lbl']
#df.drop('hab_lbl', axis=1, inplace=True)

In [4]:
from sklearn.preprocessing import LabelEncoder

In [5]:
cat_vars = df.columns[np.where(df.dtypes == 'int64')]
cat_vars = cat_vars.tolist()
cat_vars.remove('hab_lbl')

In [6]:
cont_vars = df.columns[np.where(df.dtypes != 'int64')].tolist()

In [7]:
label_encoders = {}

In [8]:
for cat_col in cat_vars:
    label_encoders[cat_col] = LabelEncoder()
    df[cat_col] = label_encoders[cat_col].fit_transform(df[cat_col])

In [9]:
cat_vars

['P. Zone Class',
 'P. Mass Class',
 'P. Composition Class',
 'P. Atmosphere Class']

In [10]:
train_df, test_df = train_test_split(df, train_size=0.7)

/home/rahul/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [11]:
train_df = rebalance(train_df)

In [12]:
train_df.drop('hab_lbl', axis=1, inplace=True)
test_df.drop('hab_lbl', axis=1, inplace=True)

/home/rahul/.local/lib/python3.5/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [13]:
train_df.head()

,P. Zone Class,P. Mass Class,P. Composition Class,P. Atmosphere Class,P. Min Mass (EU),P. Mass (EU),P. Radius (EU),P. Density (EU),P. Gravity (EU),P. Esc Vel (EU),...,S. Appar Mag,S. Mag from Planet,S. Size from Planet (deg),S. Hab Zone Min (AU),S. Hab Zone Max (AU),P. HZD,P. HZC,P. HZA,P. HZI,P. ESI
1127,2,1,1,2,18.217914,4.42,1.60,1.08,1.72,1.66,...,13.100000,-31.7,4.8373,0.905,2.119,-2.28,-0.15,-0.59,0.30,0.29
981,2,1,1,2,18.217914,7.92,1.87,1.21,2.26,2.06,...,14.900000,-33.6,11.6312,0.983,2.302,-2.41,-0.14,-0.60,0.29,0.26
658,2,1,1,2,18.217914,20.80,2.36,1.58,3.72,2.97,...,13.747447,-31.5,4.4998,1.000,2.352,-2.25,-0.12,0.36,0.30,0.25
512,2,1,1,2,18.217914,4.09,1.57,1.06,1.66,1.62,...,13.747447,-28.1,2.6373,0.150,0.387,-1.44,-0.15,-0.06,0.41,0.57
1533,2,3,1,3,18.217914,0.63,0.91,0.84,0.77,0.83,...,12.700000,-34.0,15.4252,0.774,1.824,-2.40,-0.18,-1.29,0.27,0.29


In [15]:
y = to_categorical(np.array(y))

In [19]:
cat_inps = [list(df[x]) for x in cat_vars]
cont_inp = list(df[cont_vars])

In [21]:
df[cont_vars].shape

(1713, 37)

In [44]:
model_cat_inps = [Input(shape=(1,)) for _ in cat_inps]
model_cont_inp = Input(shape=(1, 37), name='cont_inp')

embeddings = [Embedding(input_dim=len(np.unique(x)),
                        output_dim=round(1.6 * len(np.unique(x)) ** 0.56)
                       )(y) for x, y in zip(cat_inps, model_cat_inps)]
bn1 = BatchNormalization(name='first_bn')(model_cont_inp)

concat = keras.layers.concatenate([*embeddings, bn1], name='concatenate')
for i in range(2):
    relu = Dense(5 * (i+1), activation='relu', name='dense' + str(i+1))(concat)
    bn = BatchNormalization()(relu)
    drop = Dropout(0.2)(bn)

out = Dense(3, activation='softmax')(drop)

model = Model(inputs=[*model_cat_inps, model_cont_inp], outputs=out)

ValueError: Axis -1 of input tensor should have a defined dimension but the layer received an input with shape (None, 1, 37, None).

In [40]:
model.compile(SGD(0.1), loss='categorical_crossentropy', metrics=['accuracy'])

In [41]:
len(cat_inps)

4

In [42]:
model.fit(x=[*cat_inps, cont_inp], y=y, epochs=10)

ValueError: Error when checking input: expected cont_inp to have 3 dimensions, but got array with shape (37, 1)